In [59]:
def load_label(image_path, label_dir_path):
    label_file_name = os.path.basename(image_path).split(".")[0] + ".txt"
    label_file_path = os.path.join(label_dir_path,label_file_name)
    with open(label_file_path, 'r') as f:
        lines = f.readlines()
    lines= [line.rstrip() for line in lines]
    lines= [line.split() for line in lines]
    lines = numpy.array(lines).astype(float)
    lines = torch.from_numpy(lines)
    lines = sorted(lines, key=lambda x: x[1])
    sorted_labels = torch.tensor([t[0] for t in lines], dtype=torch.float64)
    sorted_boxes = [t[1:] for t in lines]
    plate_info = { "sorted_labels": sorted_labels, "sorted_boxes_xywhn":sorted_boxes}
    return plate_info

In [61]:
import os
lines = load_label(r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp\2a602f33-112d-4003-9580-cdb52dbdc245_52_737_gt_3281721_roi_3281721_151.jpg",r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp_labels")

In [53]:
lines

{'sorted_labels': tensor([ 3., 18.,  2., 23.,  8., 13., 18.,  1., 10.,  7., 23.,  2., 10.,  1.],
        dtype=torch.float64),
 'sorted_boxes_xywhn': [tensor([0.2407, 0.4377, 0.0825, 0.3378], dtype=torch.float64),
  tensor([0.3193, 0.1722, 0.0264, 0.1111], dtype=torch.float64),
  tensor([0.3329, 0.4566, 0.0816, 0.3311], dtype=torch.float64),
  tensor([0.3743, 0.1761, 0.0676, 0.1122], dtype=torch.float64),
  tensor([0.4207, 0.4716, 0.0830, 0.3389], dtype=torch.float64),
  tensor([0.4554, 0.1922, 0.0716, 0.1178], dtype=torch.float64),
  tensor([0.5061, 0.2056, 0.0297, 0.1133], dtype=torch.float64),
  tensor([0.5083, 0.4916, 0.0878, 0.3344], dtype=torch.float64),
  tensor([0.5639, 0.2150, 0.0682, 0.1167], dtype=torch.float64),
  tensor([0.6032, 0.5021, 0.0818, 0.3378], dtype=torch.float64),
  tensor([0.6422, 0.2256, 0.0682, 0.1222], dtype=torch.float64),
  tensor([0.6880, 0.5221, 0.0878, 0.3422], dtype=torch.float64),
  tensor([0.7122, 0.2439, 0.0635, 0.1211], dtype=torch.float64),
  tens

In [ ]:
from ultralytics import YOLO
import cv2
from PIL import Image
import torchvision.transforms as T
import os

model = YOLO(r"D:\Users\r2shaji\Downloads\ocr_detection_model_v7_sz512_20241102\ocr_detection_model_v7_sz512_20241102.onnx")  
image = cv2.imread(r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp\2a602f33-112d-4003-9580-cdb52dbdc245_52_737_gt_3281721_roi_3281721_151.jpg")
fake_B_PlateNum = model.predict(image)

plate_info = load_label(r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp\2a602f33-112d-4003-9580-cdb52dbdc245_52_737_gt_3281721_roi_3281721_151.jpg",r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\label")
sorted_fake_B_PlateNum = sort_bbox(fake_B_PlateNum[0].boxes.xywhn)


print('self.plate_info["sorted_boxes_xywhn"]:',plate_info["sorted_boxes_xywhn"])
print('sorted_fake_B_PlateNum:',sorted_fake_B_PlateNum)


loss_CIoU_Plate = get_loss(sorted_fake_B_PlateNum,plate_info["sorted_boxes_xywhn"])
print(loss_CIoU_Plate)

In [43]:
import torch
import numpy
lines = np.array(lines).astype(float)
lines = torch.from_numpy(lines)

In [37]:
sorted(lines, key=lambda x: x[1])

[tensor([3.0000, 0.2407, 0.4377, 0.0825, 0.3378], dtype=torch.float64),
 tensor([18.0000,  0.3193,  0.1722,  0.0264,  0.1111], dtype=torch.float64),
 tensor([2.0000, 0.3329, 0.4566, 0.0816, 0.3311], dtype=torch.float64),
 tensor([23.0000,  0.3743,  0.1761,  0.0676,  0.1122], dtype=torch.float64),
 tensor([8.0000, 0.4207, 0.4716, 0.0830, 0.3389], dtype=torch.float64),
 tensor([13.0000,  0.4554,  0.1922,  0.0716,  0.1178], dtype=torch.float64),
 tensor([18.0000,  0.5061,  0.2056,  0.0297,  0.1133], dtype=torch.float64),
 tensor([1.0000, 0.5083, 0.4916, 0.0878, 0.3344], dtype=torch.float64),
 tensor([10.0000,  0.5639,  0.2150,  0.0682,  0.1167], dtype=torch.float64),
 tensor([7.0000, 0.6032, 0.5021, 0.0818, 0.3378], dtype=torch.float64),
 tensor([23.0000,  0.6422,  0.2256,  0.0682,  0.1222], dtype=torch.float64),
 tensor([2.0000, 0.6880, 0.5221, 0.0878, 0.3422], dtype=torch.float64),
 tensor([10.0000,  0.7122,  0.2439,  0.0635,  0.1211], dtype=torch.float64),
 tensor([1.0000, 0.7688, 0.53

In [137]:
def select_indices_target_boxes(pred_boxes, true_boxes, ciou_threshold=0.3):

        max_loss = 1
        if len(pred_boxes) == 0:
            return max_loss

        pred_boxes = torch.stack(pred_boxes)
        true_boxes = torch.stack(true_boxes)
        pred_boxes = pred_boxes.float().to(pred_boxes.device)
        true_boxes = true_boxes.float().to(true_boxes.device)

        matched_indices = {}

        for idx, pred_box in enumerate(pred_boxes):
            ciou_scores = bbox_iou(pred_box,true_boxes, xywh=True, CIoU=True)
            best_ciou= ciou_scores.max()
            matched_idx = torch.argmax(ciou_scores)
            print(matched_idx)
            if best_ciou > ciou_threshold:
                matched_indices[idx] = matched_idx.item()

        return matched_indices

In [139]:
matched_indices = select_indices_target_boxes(sorted_fake_B_PlateNum,plate_info["sorted_boxes_xywhn"])
matched_indices

tensor(0)
tensor(2)
tensor(4)
tensor(7)
tensor(9)
tensor(11)
tensor(13)


{0: 0, 1: 2, 2: 4, 3: 7, 4: 9, 5: 11, 6: 13}

In [147]:
def find_corresponding_correct_label(matched_indices,sorted_labels):
    relevant_indices = list(matched_indices.values())
    relevant_values = sorted_labels[relevant_indices]
    return relevant_values

In [155]:
print(list(matched_indices.values()))
find_corresponding_correct_label(matched_indices,plate_info["sorted_labels"])

[0, 2, 4, 7, 9, 11, 13]


tensor([3., 2., 8., 1., 7., 2., 1.], dtype=torch.float64)

In [165]:
def select_relevant_log_prob(sorted_class_log_prob,matched_indices):
        relevant_indices = list(matched_indices.keys())
        relevant_values = sorted_class_log_prob[relevant_indices]
        return relevant_values

In [ ]:
select_relevant_log_prob(sorted_class_log_prob,matched_indices)

In [ ]:
import os
from PIL import Image

def resize_images(input_folder, output_folder, size=(256, 256)):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output directory: {output_folder}")

    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    save_params = {}
    save_params['quality'] = 100
    save_params['subsampling'] = 0
    save_params['optimize'] = False

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(supported_extensions):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            try:
                with Image.open(input_path) as img:
                    # Resize the image
                    resized_img = img.resize(size)
                    # Save the resized image to the output folder
                    resized_img.save(output_path, **save_params)
                    print(f"Resized and saved: {filename}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")


input_dir = r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp"
output_dir = r"D:\Users\r2shaji\Downloads\lpdata\ocr_merged\train\sharp_resized"
resize_images(input_dir, output_dir, size=(256, 256))
